<H1> Extract</H1>
Extract GameBus-Experiencer user data

This Notebook handles monitoring data (incl. GPS location, activity type, heartrate, and accelerometer) and EMA data (incl. notifications and mood survey data)

In [55]:
import json  
import requests  
import pandas as pd
import datetime
import ast
import os
from   collections import defaultdict
#Import secrete variables containing or leading to sensible information
from secret.users import GB_users_path
from secret.auth import authcode
from secret.output import output_path

<h5><b>Load the account credentials of GameBus users (hereafter, players) from a csv file</b></h5>
<p>The csv file should include the username/email (column 'Username') and password (column 'Password') of the participants Gamebus account. Columns must have a header.</p> 
<p><i> See example file 'GB-users.csv'</i></p>
<p>

In [2]:
df    = pd.read_csv(GB_users_path, delimiter=';')
users = df.to_numpy()
print(f"Users successfuly loaded: {len(df)}")

Users successfuly loaded: 2


<h4><b>Define API endpoints</b></h4>
<p>Note what GameBus server you are working with for the base_url</p>

In [3]:
base_url       = "https://api-new.gamebus.eu/v2" # "https://api.healthyw8.gamebus.eu/v2"
player_id_url  = base_url + "/users/current"
token_url      = base_url + "/oauth/token"
activities_url = base_url + "/players/{}/activities?sort=-date"

<h4><b>Define auxiliary functions</b></h4>

In [4]:
def get_player_token(username, password, authcode, token_url):
    payload = {
        "grant_type": "password",
        "username"  : username,
        "password"  : password
    }
    headers = {
        "Authorization": f"Basic {authcode}",
        "Content-Type" : "application/x-www-form-urlencoded",
    }
    try:
        response = requests.post(token_url, headers=headers, data=payload)
        response.raise_for_status()  
        token = response.json().get("access_token")
        print("Token fetched successfully")
        return token
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [5]:
def get_player_id(username, password, token, player_id_url):
    payload = {
        "grant_type": "password",
        "username"  : username,
        "password"  : password
    }
    headers = {
        "Authorization": f"Bearer {token}"
    }
    try:
        response = requests.get(player_id_url, headers=headers, params=payload)
        response.raise_for_status()  # Ensure we raise an error for bad responses
        player_data = response.json()
        player_id = player_data.get("player", {}).get("id")
        print("Player_id fetched successfully")
        return player_id
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [27]:
def get_player_data(token, player_id, activities_url, gameDescriptor_tk):
    
    valid_gameDescriptor_tk = ["GEOFENCE", "LOG_MOOD", "TIZEN(DETAIL)", "NOTIFICATION(DETAIL)", "SELFREPORT"]
    if gameDescriptor_tk not in valid_gameDescriptor_tk:
        raise ValueError("Invalid gameDescriptor_tk provided. Valid descriptors are: " + ", ".join(valid_gameDescriptor_tk))

    headers = {"Authorization": f"Bearer {token}"}
    if gameDescriptor_tk == "SELFREPORT":
        data_url = (activities_url + "&excludedGds=").format(player_id)
    else:    
        data_url = (activities_url + "&gds={}").format(player_id, gameDescriptor_tk)
        
    all_player_data = []
    page = 0
    while True:
        paginated_url = f"{data_url}&page={page}"
        try:
            response = requests.get(paginated_url, headers=headers)
            response.raise_for_status()
            print(f"Player data fetched successfully for page {page}")
            player_data = response.json()
            if not player_data:
                break
            all_player_data.extend(player_data)
            page += 1
        except requests.exceptions.RequestException as e:
            print(f"Failed to retrieve data: {e}")
            break
    print(f"Total data points fetched: {len(all_player_data)}")
    return all_player_data

In [ ]:
def parse_player_data(player_data, gameDescriptor_tk, property_key="UNKNOWN"):

    #IF gameDescriptor_tk == "TIZEN(DETAIL)", please specify the property key
    
    valid_gameDescriptor_tk = ["GEOFENCE", "LOG_MOOD", "TIZEN(DETAIL)", "NOTIFICATION(DETAIL)", "SELFREPORT"]
    if gameDescriptor_tk not in valid_gameDescriptor_tk:
        raise ValueError("Invalid gameDescriptor_tk provided. Valid descriptors are: " + ", ".join(valid_gameDescriptor_tk))
    
    valid_property_key = ["UNKNOWN", "ACTIVITY_TYPE", "HRM_LOG", "ACC"]
    if property_key not in valid_property_key:
        raise ValueError("Invalid property_key provided. Valid descriptors are: " + ", ".join(valid_property_key))

    if gameDescriptor_tk == "GEOFENCE" or gameDescriptor_tk == "LOG_MOOD" or gameDescriptor_tk == "SELFREPORT" or gameDescriptor_tk == "NOTIFICATION(DETAIL)":
        data_list = []
        for player_data_point in player_data:
            data = {}
            for property_instance in player_data_point["propertyInstances"]:
                property_key = property_instance["property"]["translationKey"]
                property_value = property_instance["value"]
                data[property_key] = property_value
            data_list.append(data)
        #for item in data_list:
        #    timestamp = int(item['TIMESTAMP'])
        #    item['TIMESTAMP'] = datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    if gameDescriptor_tk == "TIZEN(DETAIL)": 
        data_list = []
        for player_data_point in player_data:
            data = {}
            for property_instance in player_data_point["propertyInstances"]:
                prop_key = property_instance["property"]["translationKey"]
                property_value = property_instance["value"]
                if prop_key == "ACTIVITY_TYPE" and property_key == "ACTIVITY_TYPE":
                    property_value_dict = ast.literal_eval(property_value)
                    data_list.append(property_value_dict)
                elif prop_key == "HRM_LOG" and property_key == "HRM_LOG": 
                    property_value_dict = ast.literal_eval(property_value)
                    for item in property_value_dict:
                        data_list.append(item)  
                elif prop_key == "ACC" and property_key == "ACC": 
                    property_value_dict = ast.literal_eval(property_value)
                    for item in property_value_dict:
                        data_list.append(item)  
        #for item in data_list:
        #    timestamp = int(item['ts'])
        #    item['ts'] = datetime.datetime.fromtimestamp(timestamp/1000).strftime('%Y-%m-%d %H:%M:%S')
    print("Data parsed succefully")
    return data_list 

In [56]:
def export_data_to_json(data_list, file_name = 'Player-data.json'):
    file_path   = f"{output_path}/{file_name}"

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    with open(file_path, 'w') as json_file:
        json.dump(data_list, json_file, indent=4)
        
    print(f"Data saved to {file_path}")
    return None

In [57]:
def export_data_to_csv(data_list, file_name='Player-data.csv'):
    file_path = f"{output_path}/{file_name}"

    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    df = pd.DataFrame(data_list)
    df.to_csv(file_path, index=False)
    
    print(f"Data saved to {file_path}")
    return None

<h4><b>Extract, parse and export data</b></h4>

Select user/ player

In [14]:
user_order = 0 # first user in the list
username   = users[user_order][0]
password   = users[user_order][1]
print(username, password)

info+bm@gamebus.nl password1234


Get player token and id

In [15]:
player_token = get_player_token(username, password, authcode, token_url)
player_id    = get_player_id(username, password, player_token, player_id_url)

Token fetched successfully
Player_id fetched successfully


<h4>Extract, parse and export <span style="color:orange;">GPS location </span> data </h4>
<p>Includes:</p>
<ul>
  <li>latitude</li>
  <li>longitude</li>
  <li>altitude</li>
  <li>speed</li>
  <li>error</li>
  <li>timestamp</li>
  <li>arm</li>
</ul>


In [ ]:
gameDescriptor_tk    = "GEOFENCE"
player_gps_data      = get_player_data(player_token, player_id, activities_url, gameDescriptor_tk)

In [63]:
player_gps_data_list = parse_player_data(player_gps_data, gameDescriptor_tk)

Data parsed succefully


In [58]:
export_data_to_json(player_gps_data_list, file_name = 'Player-data-GPS.json')
export_data_to_csv(player_gps_data_list, file_name = 'Player-data-GPS.csv')

Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-GPS.json
Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-GPS.csv


<h4>Extract, parse and export <span style="color:orange;">mood survey </span> data </h4>
<p>Includes:</p>
<ul>
  <li>valence_state_value</li>
  <li>arousal_state_value</li>
  <li>stress_state_value</li>
  <li>event_timestamp</li>
</ul>


In [65]:
gameDescriptor_tk = "LOG_MOOD" 
player_mood_data = get_player_data(player_token, player_id, activities_url, gameDescriptor_tk)

Player data fetched successfully for page 0
Player data fetched successfully for page 1
Total data points fetched: 11


In [67]:
player_mood_data_list = parse_player_data(player_mood_data, gameDescriptor_tk)

Data parsed succefully


In [68]:
export_data_to_json(player_mood_data_list, file_name = 'Player-data-mood.json')
export_data_to_csv(player_mood_data_list, file_name = 'Player-data-mood.csv')

Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-mood.json
Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-mood.csv


<h4>Extract, parse and export <span style="color:orange;">activity</span> data. </h4>
<p>Includes:
<ul>
  <li>src</li>
  <li>ts (timestamp)</li>
  <li>type</li>
  <li>speed</li>
  <li>steps (daily cummulative)</li>
  <li>walks</li>
  <li>runs</li>
  <li>freq</li>
  <li>distance</li>
  <li>cals</li>
</ul>
</p>

In [ ]:
gameDescriptor_tk = "TIZEN(DETAIL)" 
property_key = "ACTIVITY_TYPE"  
player_activity_data = get_player_data(player_token, player_id, activities_url, gameDescriptor_tk)

Player data fetched successfully for page 0
Player data fetched successfully for page 1
Player data fetched successfully for page 2
Total data points fetched: 914


In [ ]:
player_activity_data_list = parse_player_data(player_activity_data, gameDescriptor_tk, property_key)

Data parsed succefully


In [73]:
export_data_to_json(player_activity_data_list, file_name = 'Player-data-activity.json')
export_data_to_csv(player_activity_data_list, file_name = 'Player-data-activity.csv')

Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-activity.json
Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-activity.csv


<h4>Extract, parse and export <span style="color:orange;">heartrate</span> data. </h4>
<p>Includes:
<ul>
  <li>ts (timestamp)</li>
  <li>hr (heartrate)</li>
  <li>pp</li>
</ul>
</p>

In [79]:
gameDescriptor_tk = "TIZEN(DETAIL)" 
property_key = "HRM_LOG" 
player_heartrate_data = get_player_data(player_token, player_id, activities_url, gameDescriptor_tk)

Player data fetched successfully for page 0
Player data fetched successfully for page 1
Player data fetched successfully for page 2
Total data points fetched: 914


In [81]:
player_heartrate_data_list = parse_player_data(player_heartrate_data, gameDescriptor_tk, property_key)

Data parsed succefully


In [83]:
export_data_to_json(player_heartrate_data_list, file_name = 'Player-data-heartrate.json')
export_data_to_csv(player_heartrate_data_list, file_name = 'Player-data-heartrate.csv')

Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-heartrate.json
Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-heartrate.csv


<h4>Extract, parse and export <span style="color:orange;">accelerometer</span> data. </h4>
<p>Includes:
<ul>
  <li>ts (timestamp)</li>
  <li>hr (heartrate)</li>
  <li>pp</li>
</ul>
</p>

In [ ]:
gameDescriptor_tk = "TIZEN(DETAIL)" 
property_key = "ACC" 
player_acc_data = get_player_data(player_token, player_id, activities_url, gameDescriptor_tk)

In [ ]:
player_acc_data_list = parse_player_data(player_acc_data, gameDescriptor_tk, property_key)

<h4>Extract, parse and export <span style="color:orange;">notification</span> data.</h4>
<p>Includes:
<ul>
  <li>action (e.g., received, read)</li>
  <li>event_timestamp (heartrate)</li>
</ul>
</p>

In [86]:
gameDescriptor_tk = "NOTIFICATION(DETAIL)" 
player_notification_data = get_player_data(player_token, player_id, activities_url, gameDescriptor_tk)

Player data fetched successfully for page 0
Player data fetched successfully for page 1
Total data points fetched: 32


In [93]:
player_notification_data_list = parse_player_data(player_notification_data, gameDescriptor_tk)

Data parsed succefully


In [ ]:
export_data_to_json(player_notification_data_list, file_name = 'Player-data-notification.json')
export_data_to_csv(player_notification_data_list, file_name = 'Player-data-notification.csv')

Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-notification.json
Data saved to C:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\secret\output/Player-data-notification.csv
